# Statistical Analysis

## Logit Regression

### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import statsmodels.api as sm
import pickle

### Retrieve data

In [2]:
df = pd.read_csv('ready_df.csv')

In [3]:
df.head()

,ammonia,arsenic,barium,cadmium,copper,fluoride,bacteria,viruses,lead,nitrates,mercury,perchlorate,radium,selenium,uranium,potability
0,9.08,0.200000,2.85,0.083666,0.412311,0.05,0.001600,0.000000,0.054,16.08,0.007,2.030803e+06,6.78,0.08,0.02,1
1,21.16,0.100000,3.31,0.044721,0.812404,0.90,0.178506,0.422500,0.100,2.01,0.003,1.083072e+06,3.21,0.08,0.05,1
2,14.02,0.200000,0.58,0.089443,0.141421,0.99,0.000006,0.000009,0.078,14.16,0.006,6.391180e+06,7.07,0.07,0.01,0
3,11.33,0.200000,2.96,0.031623,1.288410,1.08,0.254117,0.504100,0.016,1.41,0.004,6.917981e+03,1.72,0.02,0.05,1
4,24.33,0.173205,0.20,0.077460,0.754983,0.61,0.000286,0.000001,0.117,6.74,0.003,8.157307e+04,2.41,0.02,0.02,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7459 entries, 0 to 7458
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ammonia      7459 non-null   float64
 1   arsenic      7459 non-null   float64
 2   barium       7459 non-null   float64
 3   cadmium      7459 non-null   float64
 4   copper       7459 non-null   float64
 5   fluoride     7459 non-null   float64
 6   bacteria     7459 non-null   float64
 7   viruses      7459 non-null   float64
 8   lead         7459 non-null   float64
 9   nitrates     7459 non-null   float64
 10  mercury      7459 non-null   float64
 11  perchlorate  7459 non-null   float64
 12  radium       7459 non-null   float64
 13  selenium     7459 non-null   float64
 14  uranium      7459 non-null   float64
 15  potability   7459 non-null   int64  
dtypes: float64(15), int64(1)
memory usage: 932.5 KB


#### Distinguish the explanatory and outcome variables

In [5]:
# Let explanatory variables be X and the outcome variable be y

X = df.iloc[:, 0:15]
y = df['potability']

#### Split data into a training set and a test set (3:1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

### Model development
#### Fit model to appropriate data

In [7]:
logit_fit = sm.GLM(y_train, X_train, family=sm.families.Binomial()).fit()
logit_fit.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
               Results: Generalized linear model
================================================================
Model:              GLM              AIC:            3292.9881  
Link Function:      logit            BIC:            -44880.7128
Dependent Variable: potability       Log-Likelihood: -1631.5    
Date:               2022-02-02 10:02 LL-Null:        -2019.9    
No. Observations:   5594             Deviance:       3263.0     
Df Model:           14               Pearson chi2:   6.39e+03   
Df Residuals:       5579             Scale:          1.0000     
Method:             IRLS                                        
----------------------------------------------------------------
               Coef.   Std.Err.    z     P>|z|   [0.025   0.975]
----------------------------------------------------------------
ammonia        -0.0075   0.0050  -1.4779 0.1394  -0.0173  0.0024
arsenic        -1.2973   0.2691  -4.8199 0.0000  -1.8248 -0.7697
barium          0.3801   0.0396   9.6071 0.0000   0.3026  0.4577
cadmium        -8.6422   0.5673 -15.2333 0.0000  -9.7541 -7.5302
copper          0.1231   0.1077   1.1429 0.2531  -0.0880  0.3343
fluoride        0.1493   0.1002   1.4893 0.1364  -0.0472  0.3457
bacteria        2.1130   0.5361   3.9413 0.0001   1.0622  3.1638
viruses        -2.3217   0.3568  -6.5068 0.0000  -3.0210 -1.6223
lead           -0.0872   0.7802  -0.1118 0.9110  -1.6163  1.4420
nitrates       -0.0238   0.0079  -3.0139 0.0026  -0.0393 -0.0083
mercury       -31.3973  14.7602  -2.1272 0.0334 -60.3267 -2.4680
perchlorate    -0.0000   0.0000  -4.5968 0.0000  -0.0000 -0.0000
radium          0.0261   0.0198   1.3181 0.1875  -0.0127  0.0650
selenium       -2.8755   1.5442  -1.8621 0.0626  -5.9021  0.1512
uranium        -9.2466   1.6485  -5.6090 0.0000 -12.4777 -6.0156
================================================================

"""

#### Drop the columns that are not statistically significant with respect to potability at the 0.05 significance level (explained in more detail below). 
#### Note that each column being dropped was determined to have no/weak correlation to potability in the initial data exploration

In [8]:
X_train.drop(['ammonia', 'copper', 'fluoride', 'lead', 'radium', 'selenium'], axis=1, inplace=True)
X_test.drop(['ammonia', 'copper', 'fluoride', 'lead', 'radium', 'selenium'], axis=1, inplace=True)

#### Fit the model again with an intercept

In [9]:
# Add constants to dataframes with explanatory variables
train_df_const = sm.add_constant(X_train)
test_df_const = sm.add_constant(X_test)

# Fit model
logit_model = sm.GLM(y_train, train_df_const, family=sm.families.Binomial()).fit()
logit_model.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
               Results: Generalized linear model
================================================================
Model:              GLM              AIC:            3283.8678  
Link Function:      logit            BIC:            -44922.9803
Dependent Variable: potability       Log-Likelihood: -1631.9    
Date:               2022-02-02 10:02 LL-Null:        -2019.9    
No. Observations:   5594             Deviance:       3263.9     
Df Model:           9                Pearson chi2:   6.93e+03   
Df Residuals:       5584             Scale:          1.0000     
Method:             IRLS                                        
----------------------------------------------------------------
              Coef.   Std.Err.    z     P>|z|   [0.025   0.975] 
----------------------------------------------------------------
const          0.5355   0.1755   3.0517 0.0023   0.1916   0.8794
arsenic       -1.3304   0.2729  -4.8754 0.0000  -1.8652  -0.7956
barium         0.3464   0.0396   8.7394 0.0000   0.2687   0.4240
cadmium       -9.5252   0.6034 -15.7849 0.0000 -10.7079  -8.3425
bacteria       2.2241   0.5304   4.1935 0.0000   1.1846   3.2636
viruses       -2.4839   0.3523  -7.0511 0.0000  -3.1743  -1.7934
nitrates      -0.0355   0.0082  -4.3251 0.0000  -0.0516  -0.0194
mercury      -48.6545  15.2251  -3.1957 0.0014 -78.4951 -18.8139
perchlorate   -0.0000   0.0000  -4.8263 0.0000  -0.0000  -0.0000
uranium      -10.8663   1.7111  -6.3506 0.0000 -14.2199  -7.5127
================================================================

"""

### Evaluating the new logit regression with metrics:

#### COEFFICIENTS: 

**Location:** column Coef in summary table above.   

**Meaning:** Each explanatory variable has a different coefficient, the value of which means that for each unit increase of the explanatory variable, the log-odds of a water resource being classified as potable increase by the coefficient value. 

For instance, the coefficient for arsenic is -1.3304. Thus, for every unit increase in arsenic (1 ppm), the log-odds of the water resource being potable decreases by a value of 1.3304.

#### INTERCEPT:

**Location:** row const in summary table above.   

**Meaning:** The value of the intercept is 0.5355, meaning that when all explanatory variables have a value of 0, the predicted log-odds of having a potable water resource is 0.5355.

#### P-VALUES:

**Location:** column P>|z| in summary table above.   

**Meaning:** The p-value for each explanatory variable shown above is less than 0.05 (the conventional level of significance), which means that the effect of these explanatory variables on potability is statistically significant at the 0.05 significance level.

That is, we can have confidence that the relationship between the explanatory variables in the logit model and potability is unlikely to be solely attributed to sampling error.

#### CONFIDENCE INTERVAL:

**Location:** range of values from columns [0.025 &emsp; 0.975] in summary table above.   

**Meaning:** The 95% confidence interval for each explanatory variable indicates the range that 95% of coefficient estimates will have. Note that the interval of one parameter, perchlorate, crosses 0. This means that perchlorate is not actually statistically significant at the 95% confidence interval (despite its low p-value) as 95% of perchlorate coefficient estimates will be 0.

### Summary of Hypothesis Testing Results Based on Logit Regression

#### Recall: Null hypothesis H<sub>0<sub>

None of the 20 predictor variables have a statistically significant relationship with the response variable, water potability.

#### Because the p-values of arsenic, barium, cadmium, bacteria, viruses, nitrates, mercury, and uranium are less than the significance level of 0.05, I reject the null hypothesis and accept the alternative hypothesis that there is statistically significant correlation between these variables and water potability.

#### This does not mean that there is not a statistically significant relationship between potability and explanatory variables that  includes the 12 variables dropped in wrangling or primary model fitting. It simply means that there is not sufficient evidence to reject the null hypothesis in those instances.

#### Recall: feature engineering was performed to meet the assumptions of logistic regression, so according to the orders given to variables, the equation that could be used to predict *p*, the probability of potability, is as follows:

![equation to predict probability of potability](./log_reg_eqn.png)

### Evaluating the logit regression with prediction:

#### Predicting test set results and getting confusion matrix:

In [10]:
# Use test dataframe to predict potability values

y_pred = logit_model.predict(test_df_const)

In [11]:
# Compare predicted potability values to actual potability values

metrics.confusion_matrix(y_test, y_pred.round(0))

array([[1616,   21],
       [ 214,   14]], dtype=int64)

#### Use confusion matrix to calculate accuracy of the model

In [12]:
# Top left and bottom right values are accurately predicted
# Bottom left and top right values are inaccurately predicted

accuracy = (1616 + 14) / (1630 + 214 + 21)
print('Accuracy of logistic regression classsifier on test set: {:.3f}'.format(accuracy))

Accuracy of logistic regression classsifier on test set: 0.874


#### 87.4% accuracy is quite accurate

#### Now compute precision, recall, F-measure, and support
#### Where precision is the ability of the classifier to label a sample positive if it is not negative, recall is the ability to find all positive samples, F-measures are the weighted mean of precision and recall, and support is the number of occurrences in each class of y_test

In [13]:
pd.DataFrame(metrics.classification_report(y_test, y_pred.round(0), output_dict=True)).T

,precision,recall,f1-score,support
0,0.883060,0.987172,0.932218,1637.000000
1,0.400000,0.061404,0.106464,228.000000
accuracy,0.873995,0.873995,0.873995,0.873995
macro avg,0.641530,0.524288,0.519341,1865.000000
weighted avg,0.824005,0.873995,0.831268,1865.000000


#### As seen in the table above, the model has a very poor ability to find all positive samples (recall for row 1), as well as a poor ability to predict a sample to be positive if it is not negative (precision for row 1)


#### This could be due to imbalanced data - the support is much higher for nonpotable samples than it is for potable samples. Data imbalance can be corrected using oversampling.

#### However, in the case of water potability, false negatives are better than false positives. Thus, while precision is a serious issue (more false positives than true positives), recall is not a serious problem (more false negatives than false/true negatives.

### Save model for later use

In [14]:
filename = 'logit_model.sav'
pickle.dump(logit_model, open(filename, 'wb'))